In [0]:
%sql
create view if not exists gold.customer_segment as(
  with base_query as(
    select 
      of.OrderID,
      of.ProductID,
      of.LastEditedWhen,
      round((of.Quantity * of.UnitPrice),2) as revenue,
      of.Quantity,
      c.skey as customer_key,
      c.CustomerID,
      concat(c.CustomerFirstName,' ',c.CustomerLastName) as customer_name
      
    from 
      gold.dim_date dd
    join
      gold.orders_fact of 
    on of.LastEditedDate = dd.date
    left join 
      gold.customers c 
    on of.CustomerID = c.CustomerID
    where 
      c.active_flg = 1
  ), customer_segment as (
  select 
    customer_key,
    CustomerID,
    customer_name,
    count(distinct OrderID) as total_orders,
    round(sum(revenue),2) as total_revenue,
    sum(Quantity) as total_quantity,
    count(distinct ProductID) as total_products,
    max(LastEditedWhen) as last_order_date,
    min(LastEditedWhen) as first_order_date,
    date_diff(MONTH, min(LastEditedWhen), max(LastEditedWhen)) as life_span

  from 
    base_query
  group by 
    customer_key,
    CustomerID,
    customer_name
  )
  select
    customer_key,
    CustomerID,
    customer_name,
    case when life_span >= 12 and total_revenue > 10000 then 'Loyal'
        when life_span >= 12 and total_revenue <= 10000 then 'Regular'
        else 'New' 
    end as customer_segment,
    last_order_date,
    datediff(month, last_order_date, getdate()) as recency,
    total_orders,
    total_revenue,
    total_quantity,
    total_products,
    life_span,
    --Average order value
    case when total_orders > 0 then
      round(total_revenue / total_orders, 2) 
      else 0
    end as avg_order_value,
    --Average monthly spend
    case when life_span > 0 then
      round(total_revenue / life_span,2) 
      else total_revenue
    end as avg_monthly_spend
  from 
    customer_segment
)



In [0]:
%sql
select * from gold.customer_segment